## TradeBuddy

This is a trading bot for the game Ragnarok Online.

### Features

- Main Features
  - Find tradeable deals with other users to be equivalent to their items worth plus zeny if need it be.
  - Buy and sell items
  - Analyze market trends
  - Make trades based on predefined strategies
  - Automatically adjust strategies based on market conditions
  - Backtest trading strategies
  - Visualize trading performance
- Additional Features
  - Item search
  - Market trend analysis
  - Price prediction
  - Trade history
  - Account management
  

### Technologies Used
 - LLM
 - Python


In [1]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import torch
import os
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
import json
from dotenv import load_dotenv
import logging
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Dict, Optional, Any # Added Optional and Any

In [2]:
# Model Options
MODEL_MISTRAL = 'mistralai/Mistral-7B-Instruct-v0.2'

In [3]:
# --- Setup Logging ---
# Configure logging to output to console
# You can customize the format, level, and output (e.g., to a file)
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s')
logger = logging.getLogger(__name__) # Create a logger for this module

logger.info(f"PyTorch version: {torch.__version__}")
logger.info(f"MPS available: {torch.backends.mps.is_available()}")
logger.info(f"MPS built: {torch.backends.mps.is_built()}")

# --- Authentication
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
if not hf_token:
    logger.warning("HF_TOKEN environment variable not found. Some operations might fail.")
else:
    try:
        login(hf_token, add_to_git_credential=True)
        logger.info("Successfully logged into Hugging Face Hub.")
    except Exception as e:
        logger.error(f"Failed to log into Hugging Face Hub: {e}")

2025-05-28 00:04:53,420 - INFO - __main__ - PyTorch version: 2.6.0
2025-05-28 00:04:53,432 - INFO - __main__ - MPS available: True
2025-05-28 00:04:53,432 - INFO - __main__ - MPS built: True


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-05-28 00:04:53,851 - WARNING - huggingface_hub._login - Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-05-28 00:04:53,852 - INFO - __main__ - Successfully logged into Hugging Face Hub.


In [4]:
@dataclass
class Item:
    """Represents an item in Ragnarok Online."""
    item_number: str
    item_name: str
    item_id: str
    item_description: str
    item_price: int
    item_type: str
    owner_name: str
    owner_id: str
    willing_to_trade: bool
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'Item':
        """Create an Item instance from a dictionary."""
        return cls(
            item_number=str(data.get('item_number', '')),
            item_name=str(data.get('item_name', '')),
            item_id=str(data.get('item_id', '')),
            item_description=str(data.get('item_description', '')),
            item_price=int(data.get('item_price', 0)),
            item_type=str(data.get('item_type', '')),
            owner_name=str(data.get('owner_name', '')),
            owner_id=str(data.get('owner_id', '')),
            willing_to_trade=bool(data.get('willing_to_trade', False))
        )

In [5]:
class TradeBuddy:
    def __init__(self, csv_path: str, model_name: str = MODEL_MISTRAL):
        self.items: List[Item] = []
        self.model = None
        self.tokenizer = None
        self.load_items(csv_path)
        self.load_model(model_name)

    def load_items(self, csv_path: str) -> None:
        """Load items from CSV file."""
        try:
            df = pd.read_csv(csv_path)
            # Clean the data
            df = df.dropna(subset=['item_name', 'item_price'])
            
            # Convert DataFrame rows to Item objects
            self.items = [Item.from_dict(row.to_dict()) for _, row in df.iterrows()]
            logger.info(f"Loaded {len(self.items)} items from {csv_path}")
        except Exception as e:
            logger.error(f"Error loading items: {e}")
            raise

    def load_model(self, model_name: str):
        """Load the LLM model and tokenizer."""
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # Set pad token to be the same as eos token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
        
        # Load model with proper configuration
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
        # Ensure model's pad token ID matches tokenizer
        self.model.config.pad_token_id = self.tokenizer.pad_token_id

    def _format_items_for_prompt(self, items: List[Item]) -> str:
        """Format a list of items into a readable string for the prompt."""
        formatted_items = []
        for item in items:
            formatted_item = (
                f"- {item.item_name} (ID: {item.item_id})\n"
                f"  Type: {item.item_type}\n"
                f"  Price: {item.item_price:,} zeny\n"
                f"  Description: {item.item_description}\n"
                f"  Owner: {item.owner_name} (ID: {item.owner_id})"
            )
            formatted_items.append(formatted_item)
        return "\n\n".join(formatted_items)

    def _generate_with_model(self, prompt: str) -> str:
        """Generate text using the loaded model."""
        try:
            # Tokenize the prompt
            inputs = self.tokenizer(prompt, return_tensors="pt", padding=True)
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

            # Generate text
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=2048,  # Increased from 512
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.2,  # Added to prevent repetitive responses
                    no_repeat_ngram_size=3   # Added to prevent repetitive phrases
                )

            # Decode the generated text
            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Remove the prompt from the generated text
            if generated_text.startswith(prompt):
                generated_text = generated_text[len(prompt):].strip()
            
            return generated_text

        except Exception as e:
            logger.error(f"Error generating text: {e}")
            raise

    def _parse_suggestions(self, generated_text: str) -> List[Dict]:
        """Parse the model's output into structured trade suggestions."""
        try:
            # Split the text into individual suggestions
            suggestions = []
            current_suggestion = {
                'items': [],
                'zeny_compensation': 0,
                'reasoning': ''
            }

            # Process each line of the generated text
            for line in generated_text.split('\n'):
                line = line.strip()
                if not line:
                    continue

                # Look for item mentions
                if line.startswith('- '):
                    item_name = line[2:].split('(')[0].strip()
                    # Find the item in our database
                    matching_items = [item for item in self.items if item.item_name.lower() == item_name.lower()]
                    if matching_items:
                        current_suggestion['items'].append(matching_items[0])

                # Look for zeny compensation
                elif 'zeny' in line.lower():
                    try:
                        zeny_amount = int(''.join(filter(str.isdigit, line)))
                        current_suggestion['zeny_compensation'] = zeny_amount
                    except ValueError:
                        pass

                # Look for reasoning
                elif line.startswith('Reason:') or line.startswith('Because:'):
                    current_suggestion['reasoning'] = line.split(':', 1)[1].strip()

                # If we find a new suggestion marker, save the current one and start a new one
                elif line.startswith('Suggestion') or line.startswith('Trade'):
                    if current_suggestion['items']:
                        suggestions.append(current_suggestion)
                        current_suggestion = {
                            'items': [],
                            'zeny_compensation': 0,
                            'reasoning': ''
                        }

            # Add the last suggestion if it has items
            if current_suggestion['items']:
                suggestions.append(current_suggestion)

            return suggestions

        except Exception as e:
            logger.error(f"Error parsing suggestions: {e}")
            raise

    def generate_trade_suggestions(self, target_item: Item) -> List[Dict]:
        """Generate trade suggestions using LLM."""
        # Prepare context about available items
        available_items = [item for item in self.items if item.willing_to_trade]
        
        # Create prompt for the model
        prompt = f"""
            Given the following items in Ragnarok Online, suggest fair trades for {target_item.item_name} (worth {target_item.item_price} zeny).

            Available items for trade:
            {self._format_items_for_prompt(available_items)}

            Consider:
            1. Item rarity and demand
            2. Market value
            3. Item type compatibility
            4. Fair zeny compensation

            Suggest trades that are fair and balanced. Include zeny compensation when needed.
        """
        # Generate suggestions using the model
        suggestions = self._generate_with_model(prompt)
        return self._parse_suggestions(suggestions)
    
def generate_all_trade_suggestions(buddy: TradeBuddy) -> None:
    """Generate trade suggestions for all available items using the LLM."""
    # Get all items willing to trade
    available_items = [item for item in buddy.items if item.willing_to_trade]
    
    # Create a comprehensive prompt for the model
    prompt = f"""
    Given the following items in Ragnarok Online, suggest fair trades between them.
    Consider item rarity, demand, market value, and type compatibility.

    Available items for trade:
    {buddy._format_items_for_prompt(available_items)}

    For each item, suggest:
    1. Possible trade combinations with other items
    2. If the item owner has several items you can include them to make the difference. 
    3. Fair zeny compensation if needed
    4. Reasoning for each trade suggestion

    Format your response as:
    Item: [Item Name]
    Suggested Trades:
    1. Trade: [Items to trade] + [Zeny amount] zeny
    Reason: [Explanation]
    2. Trade: [Items to trade] + [Zeny amount] zeny
    Reason: [Explanation]

    Continue this format for each item.
    """

In [6]:
def display_trade_suggestions(buddy: TradeBuddy, target_item: Item) -> None:
    """Display trade suggestions in a markdown format."""
    suggestions = buddy.generate_trade_suggestions(target_item)
    
    print(f"# Trade Suggestions for {target_item.item_name}")
    print(f"\n## Item Details")
    print(f"- **Name:** {target_item.item_name}")
    print(f"- **Price:** {target_item.item_price:,} zeny")
    print(f"- **Type:** {target_item.item_type}")
    print(f"- **Description:** {target_item.item_description}")
    
    print("\n## Suggested Trades")
    for i, suggestion in enumerate(suggestions, 1):
        print(f"\n### Suggestion {i}")
        print("**Items to Trade:**")
        for item in suggestion['items']:
            print(f"- {item.item_name} ({item.item_price:,} zeny)")
        print(f"**Zeny Compensation:** {suggestion['zeny_compensation']:,} zeny")
        print(f"**Reasoning:** {suggestion['reasoning']}")

# Example usage
if __name__ == "__main__":
    # Initialize TradeBuddy
    buddy = TradeBuddy(
        csv_path="./data/ragnarok_items_latest.csv",
        model_name=MODEL_MISTRAL
    )
    
    # Get the first item as an example
    if buddy.items:
        target_item = buddy.items[2]
        display_trade_suggestions(buddy, target_item)

2025-05-28 00:04:53,877 - INFO - __main__ - Loaded 12 items from ./data/ragnarok_items_latest.csv


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Trade Suggestions for Celestial Scepter

## Item Details
- **Name:** Celestial Scepter
- **Price:** 1,000,000 zeny
- **Type:** Armor
- **Description:** An enchanted shield shaped like the mythical Phoenix, granting protection against fire attacks.

## Suggested Trades
